Neste notebook importarei os dados do site basketball reference(https://www.basketball-reference.com/) para criar um dataset a partir de dados da  temporada regular da NBA desde a temporada 1984-1985(Quando Michael Jordan entrou como novato) até o fim da temporada 2020-2021
para analisar como seria a tabela de classificação da temporadas regulares de 2010-2011(Lebron entra no miami heat) até o fim da temporada 2017-2018(Lebron entra no Lakers),caso Michael Jordan entrasse na NBA na temporada 2010-2011

(Lembrar de mudar isso )

In [5]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'}

times = {'Los Angeles Lakers' : 'LAL',
        'Milwaukee Bucks' : 'MIL',
        'Boston Celtics' : 'BOS',
        'Philadelphia 76ers' : 'PHI',
        'Portland Trail Blazers' : 'POR',
        'Detroit Pistons' : 'DET',
        'Denver Nuggets' : 'DEN',
        'Dallas Mavericks' : 'DAL',
        'Houston Rockets' : 'HOU',
        'San Antonio Spurs' : 'SAS',
        'New Jersey Nets' : 'NJN',
        'Utah Jazz' : 'UTA',
        'Golden State Warriors' : 'GSW',
        'Chicago Bulls' : 'CHI',
        'Cleveland Cavaliers' : 'CLE',
        'Indiana Pacers' : 'IND',
        'Phoenix Suns': 'PHO',
        'Los Angeles Clippers' : 'LAC',
        'Atlanta Hawks' : 'ATL',
        'Washington Bullets' : 'WSB',
        'New York Knicks' : 'NYK',
        'Seattle SuperSonics' : 'SEA',
        'Toronto Raptors' : 'TOR',
        'Miami Heat' : 'MIA',
        'Brooklyn Nets' : 'BRK',
        'Charlotte Hornets' : 'CHO',
        'Washington Wizards' : 'WAS',
        'Orlando Magic' : 'ORL',
        'Oklahoma City Thunder' : 'OKC',
        'Memphis Grizzlies' : 'MEM',
        'Sacramento Kings' : 'SAC',
        'New Orleans Pelicans' : 'NOP',
        'Minnesota Timberwolves' : 'MIN',
        'San Diego Clippers' : 'SDC',
        'Kansas City Kings' : 'KCK'}

anos = ['2020']

todos_jogadores = []
todos_mvps = []

## Funções para pegar os dados de mvps e os dados dos times

In [6]:
def pegar_mvps(url, dados_mvps):
    
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    mvps_html = soup.find('tbody').findAll('tr')
    
    for mvp_html in mvps_html:

        dados_mvp = {}
        dados_mvp['rank'] = int (mvp_html.find('th').getText())
        
        nome_jogador = mvp_html.find('td', {'data-stat':'player'}).find('a').getText()
        dados_mvp['team'] = mvp_html.find('td', {'data-stat':'team_id'}).getText()
        dados_mvp['award_share'] = float(mvp_html.find('td', {'data-stat':'award_share'}).getText())
        
        dados_mvps[nome_jogador] = dados_mvp


def pegar_times_conferencia(soup, dados_times, sufixo):
    times_html = soup.find('table', id = f'confs_standings_{sufixo}').find('tbody').findAll('tr')

    cont = 1
    for time_html in times_html:
        dados_time = {}
        nome_time = times[time_html.find('th').find('a').getText()]
        
        
        dados_time['conf_rank'] = cont
        cont+=1
        
        time_stats_html = time_html.findAll('td')
        
        for stat in time_stats_html:
            try:
                dados_time[stat.get('data-stat')] = float(stat.getText())
            except ValueError:
                dados_time[stat.get('data-stat')] = 0
    
        dados_times[nome_time] = dados_time
        
def pegar_times(url,dados_times):
    try:
        req = Request(url_dados_time, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    
    
    "Pegando os dados da Conferência Leste"
    pegar_times_conferencia(soup,dados_times, 'E')
    "Pegando os dados da Conferência Oeste"
    pegar_times_conferencia(soup,dados_times, 'W')
    

## Funções para carregar dados dos jogadores

In [7]:
def primeiro_scraping_jogadores(url, dados_jogadores, ano):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
   
    i = 0
    for jogador_html in jogadores_html:
        dados_jogador = {}
        if jogador_html.get('class')[0] != 'thead':
            
            dados_jogador['player'] = jogador_html.find('td', {'data-stat':'player'}).find('a').getText()
            dados_jogador['season'] = ano
            dados_jogador['pos'] = jogador_html.find('td', {'data-stat':'pos'}).getText()
            dados_jogador['age'] = int( jogador_html.find('td', {'data-stat':'age'}).getText() )
            
            if jogador_html.find('td', {'data-stat':'team_id'}).a == None:
                dados_jogador['team'] = jogador_html.find('td', {'data-stat':'team_id'}).getText()
            else:
                dados_jogador['team'] = jogador_html.find('td', {'data-stat':'team_id'}).a.getText()
            
            for stat in jogador_html.findAll('td')[4:]:
                
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())
                    
                except ValueError:
                    if(stat.getText() != ''):
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    else:
                        dados_jogador[stat.get('data-stat')] = 0.0
            dados_jogador['mp_pergame'] = round(dados_jogador['mp']/dados_jogador['g'], 1)
            dados_jogadores.append(dados_jogador)
        else:
            i+=1
            
    print(f'Teste na primeira função:{i}')
        
def intermedirario_scraping_jogadores(url, dados_jogadores):
    try:
        req = Request(url, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    jogadores_html = soup.find('tbody').findAll('tr')
    
    i = 0
    
    for jogador_html in jogadores_html:
        dados_jogador = dados_jogadores[i]
        
        if jogador_html.get('class')[0] != 'thead':
            i += 1
            
            stats_jogador = jogador_html.findAll('td')[8:]
            for stat in stats_jogador:
                try:
                    dados_jogador[stat.get('data-stat')] = int(stat.getText())

                except ValueError:
                    if(stat.getText() == ''):
                        dados_jogador[stat.get('data-stat')] = 0.0
                    elif stat.getText()[0] == '-':
                        dados_jogador[stat.get('data-stat')] =(-1)*float('0' + stat.getText()[1:])
                    else:
                        dados_jogador[stat.get('data-stat')] = float('0' + stat.getText())
                    
            

In [9]:
for ano in anos:
    "Nesta seção do código serão carregadas as páginas html"
    
    url_dados_time = f'https://www.basketball-reference.com/leagues/NBA_{ano}_standings.html'
    url_dados_totais_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_totals.html'
    url_dados_por_jogo_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_game.html'
    url_dados_por_36min_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_minute.html'
    url_dados_por_100pos_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_poss.html'
    url_dados_avancados_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_advanced.html'
    url_votacao_mvp = f'https://www.basketball-reference.com/awards/awards_{ano}.html'
    
    
    "Aqui será carregado os dados dos candidatos a MVP relativos a este ano"
    
    dados_mvps = {}
    pegar_mvps(url_votacao_mvp,dados_mvps)
    
   
    
    "Aqui serão carregados os dados do time"
    
    "Aqui é a variável que armazenará os dados do time relativos àquele ano"
    dados_times = {}
    
    pegar_times(url_dados_time, dados_times)
    
    
    "Finalmente, serão carregados os dados dos jogadores"
    
    dados_jogadores = []
    
    primeiro_scraping_jogadores(url_dados_totais_jogador, dados_jogadores, int(ano))
    
    intermedirario_scraping_jogadores(url_dados_por_jogo_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_36min_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_por_100pos_jogador, dados_jogadores)
    intermedirario_scraping_jogadores(url_dados_avancados_jogador, dados_jogadores)
    
    print(dados_mvps)

    
    

Teste na primeira função:26
{'Giannis Antetokounmpo': {'rank': 1, 'team': 'MIL', 'award_share': 0.952}, 'LeBron James': {'rank': 2, 'team': 'LAL', 'award_share': 0.746}, 'James Harden': {'rank': 3, 'team': 'HOU', 'award_share': 0.363}, 'Luka Dončić': {'rank': 4, 'team': 'DAL', 'award_share': 0.198}, 'Kawhi Leonard': {'rank': 5, 'team': 'LAC', 'award_share': 0.166}, 'Anthony Davis': {'rank': 6, 'team': 'LAL', 'award_share': 0.081}, 'Chris Paul': {'rank': 7, 'team': 'OKC', 'award_share': 0.026}, 'Damian Lillard': {'rank': 8, 'team': 'POR', 'award_share': 0.023}, 'Nikola Jokić': {'rank': 9, 'team': 'DEN', 'award_share': 0.018}, 'Pascal Siakam': {'rank': 10, 'team': 'TOR', 'award_share': 0.017}, 'Jimmy Butler': {'rank': 11, 'team': 'MIA', 'award_share': 0.009}, 'Jayson Tatum': {'rank': 12, 'team': 'BOS', 'award_share': 0.001}}
